# **Chapter 38: Plugin Development**

---

## **Learning Objectives**

By the end of this chapter, you will be able to:

- Understand the Flutter plugin architecture and platform channel communication
- Create custom Flutter plugins using the federated approach
- Implement platform-specific code for Android (Kotlin/Java) and iOS (Swift/Objective-C)
- Design robust platform interfaces for multi-platform plugin development
- Publish packages to pub.dev following industry best practices
- Maintain backward compatibility while evolving plugin APIs
- Test plugins using integration tests and platform-specific unit tests

---

## **Prerequisites**

- Completed Chapter 23: Platform Channels (understanding of MethodChannel and EventChannel)
- Proficiency in Dart and at least one platform-native language (Kotlin/Java or Swift/Objective-C)
- Understanding of async/await patterns and error handling in Dart
- Familiarity with Android Studio and Xcode build systems
- Basic knowledge of Gradle (Android) and CocoaPods (iOS) dependency management

---

## **38.1 Introduction to Plugin Development**

Flutter plugins enable you to access platform-specific APIs and hardware capabilities that aren't available through the Flutter framework itself. While Flutter provides a rich set of widgets and Dart APIs, many features require native platform code: cameras, sensors, battery status, Bluetooth, AR/VR capabilities, and proprietary SDKs.

### **Understanding Plugin Architecture**

A Flutter plugin consists of two main parts:
1. **Dart API**: The public interface that Flutter developers use
2. **Platform Implementation**: Native code (Android/iOS/Desktop) that communicates with the OS

```dart
// High-level architecture of a plugin
// lib/battery_plugin.dart (Dart side)
import 'dart:async';
import 'package:flutter/services.dart';

class BatteryPlugin {
  // MethodChannel serves as the communication bridge
  static const MethodChannel _channel = 
      MethodChannel('com.example.battery_plugin');
  
  // Public API hides platform complexity from users
  static Future<int> getBatteryLevel() async {
    try {
      final int batteryLevel = await _channel.invokeMethod('getBatteryLevel');
      return batteryLevel;
    } on PlatformException catch (e) {
      throw BatteryException('Failed to get battery level: ${e.message}');
    }
  }
}

// Custom exception for plugin-specific errors
class BatteryException implements Exception {
  final String message;
  BatteryException(this.message);
  
  @override
  String toString() => 'BatteryException: $message';
}
```

**Explanation:**

- **MethodChannel**: Acts as the named communication conduit between Dart and native platforms. The string `'com.example.battery_plugin'` is a unique identifier that must match on both sides.
- **invokeMethod**: Sends a message to the platform side asynchronously. The string `'getBatteryLevel'` is the method identifier that the native side listens for.
- **PlatformException**: Catches errors thrown by the native platform (e.g., permissions denied, API unavailable).
- **Abstraction layer**: The Dart API wraps platform channels in type-safe methods, returning `Future<int>` rather than raw platform messages.

### **When to Create a Plugin**

Create a plugin when:
- You need to expose platform-specific hardware (sensors, biometrics, specialized cameras)
- Integrating third-party native SDKs (payment gateways, analytics, ML kits)
- Reusing existing native codebases within Flutter projects
- Providing optimized platform-specific implementations for performance-critical operations

**Industry Standard**: Always check [pub.dev](https://pub.dev) before creating a new plugin. Reuse existing well-maintained plugins (camera, geolocator, local_auth) rather than reinventing, but create custom plugins for proprietary business logic.

---

## **38.2 Creating a Custom Flutter Plugin**

Flutter provides tooling to scaffold plugin projects with platform-specific boilerplate.

### **Setting Up the Plugin Project**

Use the `flutter` CLI to create a plugin with federated structure support:

```bash
# Create a new plugin project with example app
flutter create --template=plugin --platforms=android,ios,macos,windows,linux --org=com.example battery_level

# Navigate to the project
cd battery_level

# Project structure:
# battery_level/
# ├── android/                 # Android platform code (Kotlin/Java)
# ├── ios/                     # iOS platform code (Swift/Objective-C)
# ├── lib/                     # Dart public API
# │   └── battery_level.dart   # Main plugin entry point
# ├── test/                    # Dart unit tests
# ├── example/                 # Example Flutter app demonstrating usage
# └── pubspec.yaml             # Plugin dependencies and metadata
```

**Explanation:**

- **`--template=plugin`**: Creates a plugin project instead of a standard Flutter app. This generates platform-specific directories and boilerplate code.
- **`--platforms`**: Specifies which platforms to support. Federated plugins can support any subset (android, ios, macos, windows, linux, web).
- **`--org`**: Reverse domain name prefix for Android package names and iOS bundle identifiers (e.g., `com.example` becomes `com.example.battery_level`).
- **`example/` directory**: Contains a runnable Flutter app that depends on your plugin. This is required for testing and pub.dev publishing.

### **Implementing the Dart API**

The Dart side defines the public contract and handles platform communication:

```dart
// lib/battery_level.dart
import 'dart:async';
import 'dart:io' show Platform;
import 'package:flutter/services.dart';
import 'package:meta/meta.dart';

/// Enum representing battery status states
enum BatteryStatus {
  /// Battery is charging
  charging,
  /// Battery is discharging
  discharging,
  /// Battery is full
  full,
  /// Battery status unknown
  unknown,
}

/// Exception thrown when battery level cannot be determined
class BatteryLevelException implements Exception {
  /// Error code from platform
  final String code;
  
  /// Human-readable error description
  final String message;
  
  /// Additional error details
  final dynamic details;

  BatteryLevelException({
    required this.code,
    required this.message,
    this.details,
  });

  @override
  String toString() => 'BatteryLevelException[$code]: $message';
}

/// A Flutter plugin for retrieving device battery information.
/// 
/// This plugin provides a unified API to access battery level and status
/// across Android and iOS platforms.
class BatteryLevel {
  /// Channel name must be unique to avoid conflicts with other plugins
  static const MethodChannel _channel = 
      MethodChannel('com.example.battery_level');
  
  /// Event channel for streaming battery status updates
  static const EventChannel _eventChannel = 
      EventChannel('com.example.battery_level/events');

  /// Private constructor to prevent instantiation
  /// This is a static utility class
  BatteryLevel._();

  /// Returns the current battery level as a percentage (0-100)
  /// 
  /// Throws [BatteryLevelException] if:
  /// - Platform doesn't support battery monitoring
  /// - Required permissions are missing
  /// - Battery service is unavailable
  static Future<int> getBatteryLevel() async {
    try {
      final int? level = await _channel.invokeMethod<int>('getBatteryLevel');
      
      if (level == null) {
        throw BatteryLevelException(
          code: 'NULL_RESULT',
          message: 'Platform returned null battery level',
        );
      }
      
      // Validate range to ensure data integrity
      if (level < 0 || level > 100) {
        throw BatteryLevelException(
          code: 'INVALID_RANGE',
          message: 'Battery level $level is outside valid range (0-100)',
        );
      }
      
      return level;
    } on PlatformException catch (e) {
      throw BatteryLevelException(
        code: e.code,
        message: e.message ?? 'Unknown platform error',
        details: e.details,
      );
    } on MissingPluginException {
      throw BatteryLevelException(
        code: 'PLUGIN_NOT_AVAILABLE',
        message: 'BatteryLevel plugin is not implemented for ${Platform.operatingSystem}',
      );
    }
  }

  /// Returns the current battery status (charging/discharging/full)
  static Future<BatteryStatus> getBatteryStatus() async {
    try {
      final String? status = await _channel.invokeMethod<String>('getBatteryStatus');
      
      return _parseBatteryStatus(status);
    } on PlatformException catch (e) {
      throw BatteryLevelException(
        code: e.code,
        message: e.message ?? 'Failed to get battery status',
      );
    }
  }

  /// Stream of battery status updates
  /// 
  /// Returns a stream that emits [BatteryStatus] whenever the battery
  /// state changes (plugged in, unplugged, full)
  static Stream<BatteryStatus> get onBatteryStatusChanged {
    return _eventChannel
        .receiveBroadcastStream()
        .map((dynamic event) => _parseBatteryStatus(event as String?))
        .handleError((error) {
      if (error is PlatformException) {
        throw BatteryLevelException(
          code: error.code,
          message: error.message ?? 'Stream error',
          details: error.details,
        );
      }
      throw error;
    });
  }

  /// Helper method to parse string status to enum
  static BatteryStatus _parseBatteryStatus(String? status) {
    switch (status) {
      case 'charging':
        return BatteryStatus.charging;
      case 'discharging':
        return BatteryStatus.discharging;
      case 'full':
        return BatteryStatus.full;
      default:
        return BatteryStatus.unknown;
    }
  }
}
```

**Explanation:**

- **MethodChannel naming**: Use reverse domain names (e.g., `com.example.battery_level`) to avoid collisions with other plugins.
- **Null safety**: Check for null results from platform code and throw descriptive exceptions rather than allowing null dereference.
- **Validation**: Validate platform data (battery range 0-100) before returning to consumers.
- **EventChannel**: Used for streaming data (battery status changes) vs. one-shot method calls.
- **Error handling**: Convert platform-specific `PlatformException` into domain-specific `BatteryLevelException` for consistent error handling across platforms.
- **Documentation**: Use Dart doc comments (`///`) to document public APIs, parameters, and thrown exceptions.

### **Android Implementation (Kotlin)**

Android implementation uses Kotlin (preferred) or Java:

```kotlin
// android/src/main/kotlin/com/example/battery_level/BatteryLevelPlugin.kt
package com.example.battery_level

import android.content.Context
import android.content.Intent
import android.content.IntentFilter
import android.os.BatteryManager
import android.os.Build
import androidx.annotation.NonNull
import io.flutter.embedding.engine.plugins.FlutterPlugin
import io.flutter.plugin.common.EventChannel
import io.flutter.plugin.common.MethodCall
import io.flutter.plugin.common.MethodChannel
import io.flutter.plugin.common.MethodChannel.MethodCallHandler
import io.flutter.plugin.common.MethodChannel.Result

/** BatteryLevelPlugin */
class BatteryLevelPlugin: FlutterPlugin, MethodCallHandler, EventChannel.StreamHandler {
  /// The MethodChannel that will handle communication between Flutter and native Android
  private lateinit var channel: MethodChannel
  private lateinit var eventChannel: EventChannel
  private var applicationContext: Context? = null
  private var eventSink: EventChannel.EventSink? = null

  /// Called when the plugin is attached to the Flutter engine
  override fun onAttachedToEngine(@NonNull flutterPluginBinding: FlutterPlugin.FlutterPluginBinding) {
    applicationContext = flutterPluginBinding.applicationContext
    
    // Setup method channel for one-shot calls
    channel = MethodChannel(flutterPluginBinding.binaryMessenger, "com.example.battery_level")
    channel.setMethodCallHandler(this)
    
    // Setup event channel for streaming
    eventChannel = EventChannel(flutterPluginBinding.binaryMessenger, "com.example.battery_level/events")
    eventChannel.setStreamHandler(this)
  }

  /// Handle incoming method calls from Dart
  override fun onMethodCall(@NonNull call: MethodCall, @NonNull result: Result) {
    when (call.method) {
      "getBatteryLevel" -> {
        val batteryLevel = getBatteryLevel()
        if (batteryLevel != -1) {
          result.success(batteryLevel)
        } else {
          result.error("UNAVAILABLE", "Battery level not available.", null)
        }
      }
      "getBatteryStatus" -> {
        val status = getBatteryStatus()
        result.success(status)
      }
      else -> {
        // Method not implemented
        result.notImplemented()
      }
    }
  }

  /// Get battery percentage using BatteryManager (API 21+) or sticky intent
  private fun getBatteryLevel(): Int {
    return if (Build.VERSION.SDK_INT >= Build.VERSION_CODES.LOLLIPOP) {
      // Modern approach using BatteryManager (API 21+)
      val batteryManager = applicationContext?.getSystemService(Context.BATTERY_SERVICE) as? BatteryManager
      batteryManager?.getIntProperty(BatteryManager.BATTERY_PROPERTY_CAPACITY) ?: -1
    } else {
      // Legacy approach using sticky intent
      val intent = IntentFilter(Intent.ACTION_BATTERY_CHANGED).let { ifilter ->
        applicationContext?.registerReceiver(null, ifilter)
      }
      
      intent?.let { batteryStatus ->
        val level = batteryStatus.getIntExtra(BatteryManager.EXTRA_LEVEL, -1)
        val scale = batteryStatus.getIntExtra(BatteryManager.EXTRA_SCALE, -1)
        if (level != -1 && scale != -1) {
          (level * 100 / scale)
        } else {
          -1
        }
      } ?: -1
    }
  }

  /// Determine battery status (charging/discharging/full)
  private fun getBatteryStatus(): String {
    val intent = IntentFilter(Intent.ACTION_BATTERY_CHANGED).let { ifilter ->
      applicationContext?.registerReceiver(null, ifilter)
    }
    
    return intent?.let { batteryStatus ->
      val status = batteryStatus.getIntExtra(BatteryManager.EXTRA_STATUS, -1)
      when (status) {
        BatteryManager.BATTERY_STATUS_CHARGING -> "charging"
        BatteryManager.BATTERY_STATUS_DISCHARGING -> "discharging"
        BatteryManager.BATTERY_STATUS_FULL -> "full"
        else -> "unknown"
      }
    } ?: "unknown"
  }

  /// EventChannel.StreamHandler implementation for streaming battery updates
  override fun onListen(arguments: Any?, events: EventChannel.EventSink?) {
    eventSink = events
    
    // In a real implementation, you would register a BroadcastReceiver here
    // to listen for ACTION_BATTERY_CHANGED and emit to eventSink
    // For brevity, we'll just emit the current status once
    eventSink?.success(getBatteryStatus())
  }

  override fun onCancel(arguments: Any?) {
    eventSink = null
    // Unregister BroadcastReceiver here in production
  }

  /// Called when the plugin is detached from the Flutter engine
  override fun onDetachedFromEngine(@NonNull binding: FlutterPlugin.FlutterPluginBinding) {
    channel.setMethodCallHandler(null)
    eventChannel.setStreamHandler(null)
    applicationContext = null
  }
}
```

**Explanation:**

- **FlutterPlugin interface**: Modern plugins implement `FlutterPlugin` for proper lifecycle management. This replaces the older `registerWith` pattern.
- **BinaryMessenger**: The communication channel between Dart and native code. Each plugin gets its own instance.
- **MethodCallHandler**: Interface for handling method invocations from Dart. Switch on `call.method` to route to appropriate handlers.
- **Result.success()**: Returns data to Dart side. Must be called exactly once per method call.
- **Result.error()**: Returns an error to Dart side with code, message, and optional details.
- **Result.notImplemented()**: Indicates this method isn't available on this platform.
- **API level checks**: Use `Build.VERSION.SDK_INT` checks to handle different Android versions gracefully.
- **Context management**: Store `applicationContext` (not Activity context) to avoid memory leaks. The plugin lifecycle is tied to the FlutterEngine, not the Activity.

### **iOS Implementation (Swift)**

iOS implementation uses Swift (preferred) or Objective-C:

```swift
// ios/Classes/BatteryLevelPlugin.swift
import Flutter
import UIKit

/// Plugin implementation for iOS
public class BatteryLevelPlugin: NSObject, FlutterPlugin {
  /// Register the plugin with the Flutter engine
  public static func register(with registrar: FlutterPluginRegistrar) {
    let channel = FlutterMethodChannel(
      name: "com.example.battery_level",
      binaryMessenger: registrar.messenger()
    )
    
    let eventChannel = FlutterEventChannel(
      name: "com.example.battery_level/events",
      binaryMessenger: registrar.messenger()
    )
    
    let instance = BatteryLevelPlugin()
    registrar.addMethodCallDelegate(instance, channel: channel)
    
    // Setup event channel
    let streamHandler = BatteryStreamHandler()
    eventChannel.setStreamHandler(streamHandler)
  }

  /// Handle method calls from Flutter
  public func handle(_ call: FlutterMethodCall, result: @escaping FlutterResult) {
    switch call.method {
    case "getBatteryLevel":
      let batteryLevel = getBatteryLevel()
      if batteryLevel != -1 {
        result(batteryLevel)
      } else {
        result(FlutterError(
          code: "UNAVAILABLE",
          message: "Battery info unavailable",
          details: nil
        ))
      }
    case "getBatteryStatus":
      result(getBatteryStatus())
    default:
      result(FlutterMethodNotImplemented)
    }
  }

  /// Retrieve battery level from UIDevice
  private func getBatteryLevel() -> Int {
    let device = UIDevice.current
    device.isBatteryMonitoringEnabled = true
    
    // Check if battery monitoring is supported
    if device.batteryState == .unknown {
      return -1
    }
    
    // Returns 0.0 to 1.0, convert to percentage
    return Int(device.batteryLevel * 100)
  }

  /// Get battery state as string
  private func getBatteryStatus() -> String {
    let device = UIDevice.current
    device.isBatteryMonitoringEnabled = true
    
    switch device.batteryState {
    case .charging:
      return "charging"
    case .full:
      return "full"
    case .unplugged:
      return "discharging"
    case .unknown:
      return "unknown"
    @unknown default:
      return "unknown"
    }
  }
}

/// Stream handler for battery status events
class BatteryStreamHandler: NSObject, FlutterStreamHandler {
  private var eventSink: FlutterEventSink?
  
  func onListen(withArguments arguments: Any?, eventSink events: @escaping FlutterEventSink) -> FlutterError? {
    self.eventSink = events
    
    // Enable battery monitoring
    UIDevice.current.isBatteryMonitoringEnabled = true
    
    // Send initial value
    sendBatteryStatus()
    
    // Register for battery state notifications
    NotificationCenter.default.addObserver(
      self,
      selector: #selector(batteryStateDidChange),
      name: UIDevice.batteryStateDidChangeNotification,
      object: nil
    )
    
    return nil
  }
  
  func onCancel(withArguments arguments: Any?) -> FlutterError? {
    eventSink = nil
    NotificationCenter.default.removeObserver(self)
    UIDevice.current.isBatteryMonitoringEnabled = false
    return nil
  }
  
  @objc private func batteryStateDidChange() {
    sendBatteryStatus()
  }
  
  private func sendBatteryStatus() {
    let device = UIDevice.current
    let status: String
    
    switch device.batteryState {
    case .charging:
      status = "charging"
    case .full:
      status = "full"
    case .unplugged:
      status = "discharging"
    default:
      status = "unknown"
    }
    
    eventSink?(status)
  }
}
```

**Explanation:**

- **FlutterPlugin protocol**: Swift plugins implement `FlutterPlugin` and register channels in `register(with:)` static method.
- **FlutterResult**: Closure that must be called exactly once with the result or error.
- **FlutterError**: Standard error type containing code, message, and details.
- **UIDevice**: iOS API for device information. Must enable `isBatteryMonitoringEnabled` before accessing battery data.
- **NotificationCenter**: iOS uses observer pattern for battery state changes. Register in `onListen`, unregister in `onCancel` to prevent memory leaks.
- **@unknown default**: Required in Swift switch statements for enums to handle future cases.

---

## **38.3 Federated Plugins Architecture**

The federated plugin architecture separates platform implementations into distinct packages, allowing for platform-specific implementations to be developed and maintained independently.

### **Federated Plugin Structure**

```
battery_level/                      # App-facing package
├── lib/
│   └── battery_level.dart          # Platform interface definition
└── pubspec.yaml

battery_level_platform_interface/     # Platform interface package
├── lib/
│   ├── battery_level_platform_interface.dart
│   └── method_channel_battery_level.dart
└── pubspec.yaml

battery_level_android/              # Android implementation
├── android/
├── lib/
│   └── battery_level_android.dart
└── pubspec.yaml

battery_level_ios/                  # iOS implementation
├── ios/
├── lib/
│   └── battery_level_ios.dart
└── pubspec.yaml
```

### **Platform Interface Package**

The platform interface defines the contract that all platform implementations must follow:

```dart
// battery_level_platform_interface/lib/battery_level_platform_interface.dart
import 'package:plugin_platform_interface/plugin_platform_interface.dart';

/// The interface that implementations of battery_level must implement.
/// 
/// This allows for platform-specific implementations (Android, iOS, etc.)
/// to exist in separate packages while maintaining a consistent API.
abstract class BatteryLevelPlatform extends PlatformInterface {
  /// Constructor marks this as the base class for the interface
  BatteryLevelPlatform() : super(token: _token);

  static final Object _token = Object();

  /// The current instance of [BatteryLevelPlatform].
  /// 
  /// Defaults to a placeholder implementation that throws UnimplementedError.
  /// Platform-specific implementations should set this to their implementation
  /// via `registerWith` method.
  static BatteryLevelPlatform _instance = _PlaceholderImplementation();

  static BatteryLevelPlatform get instance => _instance;

  /// Platform-specific implementations should set this to their implementation
  /// by calling this setter.
  static set instance(BatteryLevelPlatform instance) {
    PlatformInterface.verifyToken(instance, _token);
    _instance = instance;
  }

  /// Returns the current battery level (0-100)
  /// 
  /// Throws [UnimplementedError] if not implemented by platform.
  Future<int> getBatteryLevel() {
    throw UnimplementedError('getBatteryLevel() has not been implemented.');
  }

  /// Returns the current battery status
  Future<BatteryStatus> getBatteryStatus() {
    throw UnimplementedError('getBatteryStatus() has not been implemented.');
  }

  /// Stream of battery status changes
  Stream<BatteryStatus> get onBatteryStatusChanged {
    throw UnimplementedError('onBatteryStatusChanged has not been implemented.');
  }
}

/// Placeholder implementation that throws UnimplementedError
class _PlaceholderImplementation extends BatteryLevelPlatform {
  @override
  Future<int> getBatteryLevel() async {
    throw UnimplementedError('Platform implementation not registered');
  }

  @override
  Future<BatteryStatus> getBatteryStatus() async {
    throw UnimplementedError('Platform implementation not registered');
  }

  @override
  Stream<BatteryStatus> get onBatteryStatusChanged {
    return Stream.error(UnimplementedError('Platform implementation not registered'));
  }
}

/// Enum for battery status
enum BatteryStatus { charging, discharging, full, unknown }
```

**Explanation:**

- **PlatformInterface**: Base class from `plugin_platform_interface` package that provides token verification to ensure only valid implementations can register.
- **_token**: Unique object used to verify that only authorized implementations can set the instance.
- **_PlaceholderImplementation**: Default implementation that throws `UnimplementedError`, ensuring developers get clear errors if they forget to include a platform implementation.
- **Abstract methods**: Define the contract that all platform implementations must fulfill.
- **Setter verification**: `PlatformInterface.verifyToken` ensures the instance being set is a valid implementation, not a random class.

### **Method Channel Implementation (Shared)**

The method channel implementation can be shared or platform-specific:

```dart
// battery_level_platform_interface/lib/method_channel_battery_level.dart
import 'dart:async';
import 'package:flutter/services.dart';
import 'battery_level_platform_interface.dart';

/// The method channel implementation of [BatteryLevelPlatform].
/// 
/// This implementation uses MethodChannel and EventChannel to communicate
/// with native platform code. It can be extended by platform-specific
/// implementations or used directly if the platform uses standard method channels.
class MethodChannelBatteryLevel extends BatteryLevelPlatform {
  /// The method channel used to interact with the native platform.
  @visibleForTesting
  final MethodChannel methodChannel = 
      const MethodChannel('com.example.battery_level');

  /// The event channel used for battery status streams.
  @visibleForTesting
  final EventChannel eventChannel = 
      const EventChannel('com.example.battery_level/events');

  @override
  Future<int> getBatteryLevel() async {
    final int? level = await methodChannel.invokeMethod<int>('getBatteryLevel');
    if (level == null) {
      throw PlatformException(
        code: 'NULL_RESULT',
        message: 'Battery level returned null',
      );
    }
    return level;
  }

  @override
  Future<BatteryStatus> getBatteryStatus() async {
    final String? status = await methodChannel.invokeMethod<String>('getBatteryStatus');
    return _parseStatus(status);
  }

  @override
  Stream<BatteryStatus> get onBatteryStatusChanged {
    return eventChannel
        .receiveBroadcastStream()
        .map((dynamic event) => _parseStatus(event as String?));
  }

  BatteryStatus _parseStatus(String? status) {
    switch (status) {
      case 'charging':
        return BatteryStatus.charging;
      case 'discharging':
        return BatteryStatus.discharging;
      case 'full':
        return BatteryStatus.full;
      default:
        return BatteryStatus.unknown;
    }
  }
}
```

### **Platform-Specific Implementation (Android)**

```dart
// battery_level_android/lib/battery_level_android.dart
import 'package:battery_level_platform_interface/battery_level_platform_interface.dart';
import 'package:flutter/services.dart';

/// Android implementation of [BatteryLevelPlatform]
class BatteryLevelAndroid extends MethodChannelBatteryLevel {
  /// Register this implementation as the default instance
  static void registerWith() {
    BatteryLevelPlatform.instance = BatteryLevelAndroid();
  }

  @override
  Future<int> getBatteryLevel() async {
    try {
      return await super.getBatteryLevel();
    } on PlatformException catch (e) {
      // Android-specific error handling or logging
      if (e.code == 'UNAVAILABLE') {
        // Try alternative method for older Android versions
        return _getBatteryLevelLegacy();
      }
      rethrow;
    }
  }

  Future<int> _getBatteryLevelLegacy() async {
    // Android-specific fallback implementation
    // In a real scenario, this might use a different channel or calculation
    return -1;
  }
}
```

**Explanation:**

- **Extends MethodChannelBatteryLevel**: Platform implementations can extend the shared method channel class or implement the interface directly.
- **registerWith()**: Static method called by the Flutter plugin registrant to register this implementation as the default.
- **Platform-specific logic**: Override methods to add platform-specific behavior (e.g., Android-specific fallbacks for older versions).
- **Registration**: The `registerWith` method is called automatically by Flutter's plugin system when the app starts.

### **App-Facing Package**

The main package that users depend on:

```dart
// battery_level/lib/battery_level.dart
export 'package:battery_level_platform_interface/battery_level_platform_interface.dart'
    show BatteryStatus, BatteryLevelPlatform;

/// Battery level API for Flutter applications.
/// 
/// This class provides a simplified API that delegates to the platform-specific
/// implementation registered via [BatteryLevelPlatform].
class BatteryLevel {
  /// Returns the current battery level (0-100)
  static Future<int> get level async {
    return BatteryLevelPlatform.instance.getBatteryLevel();
  }

  /// Returns the current battery status
  static Future<BatteryStatus> get status async {
    return BatteryLevelPlatform.instance.getBatteryStatus();
  }

  /// Stream of battery status changes
  static Stream<BatteryStatus> get onStatusChanged {
    return BatteryLevelPlatform.instance.onBatteryStatusChanged;
  }
}
```

**Explanation:**

- **Export**: Re-exports types from the platform interface so users don't need to add multiple dependencies.
- **Static delegation**: Provides a clean static API that delegates to the platform instance. Users call `BatteryLevel.level` without worrying about platform specifics.
- **Separation of concerns**: The app-facing package contains no platform code, only the high-level API.

---

## **38.4 Platform Interface Design**

Designing robust platform interfaces requires careful consideration of API stability, versioning, and error handling.

### **API Design Principles**

```dart
// platform_interface/lib/src/types.dart

/// Represents battery information snapshot
/// 
/// Immutable data class for battery information
class BatteryInfo {
  /// Battery level percentage (0-100)
  final int level;
  
  /// Current charging status
  final BatteryStatus status;
  
  /// Whether the battery is low (< 20%)
  final bool isLow;
  
  /// Temperature in Celsius (if available)
  final double? temperature;
  
  /// Voltage in millivolts (if available)
  final int? voltage;
  
  /// Technology type (Li-ion, etc.) if available
  final String? technology;

  const BatteryInfo({
    required this.level,
    required this.status,
    this.isLow = false,
    this.temperature,
    this.voltage,
    this.technology,
  });

  /// Create from platform map (defensive parsing)
  factory BatteryInfo.fromMap(Map<dynamic, dynamic> map) {
    return BatteryInfo(
      level: map['level'] as int,
      status: BatteryStatus.fromString(map['status'] as String),
      isLow: map['isLow'] as bool? ?? false,
      temperature: map['temperature'] as double?,
      voltage: map['voltage'] as int?,
      technology: map['technology'] as String?,
    );
  }

  Map<String, dynamic> toMap() {
    return {
      'level': level,
      'status': status.name,
      'isLow': isLow,
      if (temperature != null) 'temperature': temperature,
      if (voltage != null) 'voltage': voltage,
      if (technology != null) 'technology': technology,
    };
  }
}

/// Enhanced enum for battery status with serialization support
enum BatteryStatus {
  charging('CHARGING'),
  discharging('DISCHARGING'),
  full('FULL'),
  unknown('UNKNOWN'),
  notCharging('NOT_CHARGING'); // Some Android devices report this

  final String platformValue;
  const BatteryStatus(this.platformValue);

  /// Parse from platform string with fallback
  static BatteryStatus fromString(String? value) {
    return BatteryStatus.values.firstWhere(
      (e) => e.platformValue == value,
      orElse: () => BatteryStatus.unknown,
    );
  }
}
```

**Explanation:**

- **Immutable data classes**: Use `final` fields and const constructors for data transfer objects.
- **Defensive parsing**: Factory constructors should handle missing or unexpected keys gracefully.
- **Optional fields**: Use nullable types for platform-specific data that might not be available on all platforms.
- **Enum serialization**: Store platform string values in enums to handle platform naming differences.
- **Backward compatibility**: Adding new optional fields doesn't break existing implementations.

### **Error Handling Strategy**

```dart
// platform_interface/lib/src/errors.dart

/// Base class for plugin exceptions
class BatteryPluginException implements Exception {
  final String code;
  final String message;
  final dynamic details;

  const BatteryPluginException({
    required this.code,
    required this.message,
    this.details,
  });

  @override
  String toString() => 'BatteryPluginException[$code]: $message';
}

/// Specific error types for better error handling
class PermissionDeniedException extends BatteryPluginException {
  const PermissionDeniedException({
    String message = 'Battery permission denied',
    dynamic details,
  }) : super(
          code: 'PERMISSION_DENIED',
          message: message,
          details: details,
        );
}

class BatteryServiceUnavailableException extends BatteryPluginException {
  const BatteryServiceUnavailableException({
    String message = 'Battery service not available on this device',
    dynamic details,
  }) : super(
          code: 'SERVICE_UNAVAILABLE',
          message: message,
          details: details,
        );
}

/// Extension to convert PlatformException to typed exceptions
extension PlatformExceptionExtension on PlatformException {
  BatteryPluginException toBatteryException() {
    switch (code) {
      case 'PERMISSION_DENIED':
        return PermissionDeniedException(message: message ?? 'Permission denied');
      case 'SERVICE_UNAVAILABLE':
        return BatteryServiceUnavailableException(message: message ?? 'Service unavailable');
      default:
        return BatteryPluginException(
          code: code,
          message: message ?? 'Unknown error',
          details: details,
        );
    }
  }
}
```

**Explanation:**

- **Exception hierarchy**: Create specific exception types for different error scenarios so users can catch specific errors.
- **Error codes**: Use consistent error codes across platforms (e.g., `PERMISSION_DENIED`).
- **Extension methods**: Convert generic `PlatformException` to domain-specific exceptions for type-safe error handling.

---

## **38.5 Publishing to pub.dev**

Publishing requires following Dart and Flutter package guidelines for quality and discoverability.

### **Package Configuration**

```yaml
# pubspec.yaml
name: battery_level
description: A comprehensive Flutter plugin for accessing battery information across Android and iOS.
version: 1.0.0
homepage: https://github.com/example/battery_level
repository: https://github.com/example/battery_level
issue_tracker: https://github.com/example/battery_level/issues

environment:
  sdk: '>=2.17.0 <4.0.0'
  flutter: ">=3.0.0"

dependencies:
  flutter:
    sdk: flutter
  battery_level_platform_interface: ^1.0.0
  
dev_dependencies:
  flutter_test:
    sdk: flutter
  flutter_lints: ^3.0.0

flutter:
  plugin:
    platforms:
      android:
        default_package: battery_level_android
      ios:
        default_package: battery_level_ios
```

**Explanation:**

- **Description**: Clear, concise description of what the package does (max 180 characters for pub.dev preview).
- **Version**: Follows semantic versioning (semver). Breaking changes require major version bump.
- **Repository links**: Essential for users to report issues and view source code.
- **Environment constraints**: Specify minimum SDK versions to ensure compatibility.
- **Plugin declaration**: The `flutter.plugin` section maps platforms to their implementation packages in federated plugins.

### **Documentation Standards**

```dart
// lib/battery_level.dart
/// A Flutter plugin for retrieving device battery information.
///
/// ## Platform Support
///
/// | Platform | Minimum Version | Features Supported |
/// |----------|----------------|-------------------|
/// | Android  | API 21+        | Level, Status, Temperature |
/// | iOS      | 11.0+          | Level, Status     |
///
/// ## Installation
///
/// Add to `pubspec.yaml`:
/// ```yaml
/// dependencies:
///   battery_level: ^1.0.0
/// ```
///
/// ## Usage
///
/// ```dart
/// import 'package:battery_level/battery_level.dart';
///
/// void main() async {
///   // Get current battery level
///   final level = await BatteryLevel.level;
///   print('Battery: $level%');
///   
///   // Listen for changes
///   BatteryLevel.onStatusChanged.listen((status) {
///     print('Status changed: $status');
///   });
/// }
/// ```
///
/// ## Permissions
///
/// ### Android
/// Add to `AndroidManifest.xml`:
/// ```xml
/// <uses-permission android:name="android.permission.BATTERY_STATS" />
/// ```
///
/// ### iOS
/// No additional permissions required.
///
/// ## Error Handling
///
/// Always wrap calls in try-catch blocks:
///
/// ```dart
/// try {
///   final level = await BatteryLevel.level;
/// } on PermissionDeniedException catch (e) {
///   // Handle permission denied
/// } on BatteryServiceUnavailableException catch (e) {
///   // Handle service unavailable
/// }
/// ```
library battery_level;

export 'package:battery_level_platform_interface/battery_level_platform_interface.dart';
// ... rest of exports
```

**Explanation:**

- **Library documentation**: Top-level doc comment for the library should include:
  - Feature matrix showing platform support
  - Installation instructions
  - Usage examples
  - Platform-specific setup requirements
  - Error handling guidance
- **Markdown tables**: Use tables for platform compatibility matrices.
- **Code blocks**: Include executable example code in documentation.

### **Publishing Checklist**

Before publishing, run these validation commands:

```bash
# Dry run to check for issues
flutter pub publish --dry-run

# Verify package scores
flutter pub get
dart analyze
dart format --set-exit-if-changed lib/

# Run tests
flutter test

# Check platform implementations
cd example
flutter build apk  # Android
flutter build ios  # iOS (requires macOS)
```

**Industry Standards:**
- **CHANGELOG.md**: Maintain a detailed changelog following [Keep a Changelog](https://keepachangelog.com/) format.
- **LICENSE**: Include an OSI-approved license (MIT, BSD, Apache 2.0).
- **README.md**: Must include badges (build status, pub version), installation, and usage.
- **Example**: The `example/` folder must build and demonstrate features.

---

## **38.6 Maintaining Backward Compatibility**

As plugins evolve, maintaining backward compatibility prevents breaking user applications.

### **Versioning Strategy**

```dart
// platform_interface/lib/battery_level_platform_interface.dart

/// Current version of the platform interface
/// 
/// Follows semantic versioning:
/// - Major: Breaking changes to interface
/// - Minor: New features, backward compatible
/// - Patch: Bug fixes
const String _platformInterfaceVersion = '2.1.0';

/// Annotation for deprecated features
@Deprecated('Use getBatteryInfo() instead. Will be removed in v3.0.0')
Future<int> getBatteryLevel() async {
  // Implementation that delegates to new method
  final info = await getBatteryInfo();
  return info.level;
}

/// New comprehensive method
Future<BatteryInfo> getBatteryInfo() async {
  throw UnimplementedError();
}
```

**Explanation:**

- **@Deprecated annotation**: Mark old methods with deprecation warnings, specifying the replacement and removal timeline.
- **Delegation**: Old methods can delegate to new implementations to avoid code duplication.
- **Semantic versioning**: Major version bumps indicate breaking changes; users pin to major versions to avoid breaks.

### **Platform Channel Versioning**

```dart
// Handle different platform implementation versions
Future<BatteryInfo> getBatteryInfo() async {
  try {
    // Try new API first
    final result = await methodChannel.invokeMethod<Map>('getBatteryInfo');
    return BatteryInfo.fromMap(result!);
  } on PlatformException catch (e) {
    if (e.code == 'NOT_IMPLEMENTED') {
      // Fall back to legacy API for older platform implementations
      final level = await methodChannel.invokeMethod<int>('getBatteryLevel');
      return BatteryInfo(
        level: level ?? -1,
        status: BatteryStatus.unknown,
      );
    }
    rethrow;
  }
}
```

**Explanation:**

- **Graceful degradation**: Try new API first, fall back to legacy if not implemented.
- **Error code checking**: Check for specific error codes to determine fallback strategy.
- **Backward compatibility**: Older platform implementations continue to work while new features are optional.

### **Testing Compatibility**

```dart
// test/battery_level_test.dart
import 'package:flutter_test/flutter_test.dart';
import 'package:flutter/services.dart';
import 'package:battery_level/battery_level.dart';

void main() {
  TestWidgetsFlutterBinding.ensureInitialized();

  group('BatteryLevel', () {
    const MethodChannel channel = MethodChannel('com.example.battery_level');
    final List<MethodCall> log = <MethodCall>[];

    setUp(() {
      TestDefaultBinaryMessengerBinding.instance.defaultBinaryMessenger
          .setMockMethodCallHandler(channel, (MethodCall methodCall) async {
        log.add(methodCall);
        switch (methodCall.method) {
          case 'getBatteryLevel':
            return 85;
          case 'getBatteryInfo':
            return {
              'level': 85,
              'status': 'charging',
              'isLow': false,
            };
          default:
            return null;
        }
      });
    });

    tearDown(() {
      log.clear();
    });

    test('getBatteryLevel returns correct value', () async {
      // Test legacy API still works
      final level = await BatteryLevel.level;
      expect(level, 85);
      expect(log.last.method, 'getBatteryLevel');
    });

    test('getBatteryInfo returns structured data', () async {
      // Test new API
      final info = await BatteryLevelPlatform.instance.getBatteryInfo();
      expect(info.level, 85);
      expect(info.status, BatteryStatus.charging);
    });
  });
}
```

**Explanation:**

- **Mocking platform channels**: Use `setMockMethodCallHandler` to simulate platform responses in unit tests.
- **Regression testing**: Test both old and new APIs to ensure backward compatibility.
- **Method call logging**: Verify that correct platform methods are invoked with expected parameters.

---

## **Chapter Summary**

In this chapter, we covered the complete lifecycle of Flutter plugin development:

### **Key Takeaways:**

1. **Plugin Architecture**: Plugins consist of a Dart API layer that communicates with platform-specific implementations via MethodChannel and EventChannel.

2. **Platform Implementation**:
   - **Android**: Implement `FlutterPlugin` interface in Kotlin/Java, handle lifecycle with `onAttachedToEngine`/`onDetachedFromEngine`
   - **iOS**: Implement `FlutterPlugin` protocol in Swift/Objective-C, use `FlutterMethodChannel` and `FlutterEventChannel`

3. **Federated Plugins**: Separate platform implementations into distinct packages (platform_interface, android, ios) to allow independent versioning and community contributions.

4. **Platform Interface Design**:
   - Use abstract classes with token verification
   - Provide placeholder implementations that throw `UnimplementedError`
   - Design immutable data classes for platform communication
   - Create specific exception types for error handling

5. **Publishing Standards**:
   - Follow semantic versioning (semver)
   - Include comprehensive README with platform matrices
   - Maintain CHANGELOG.md and proper LICENSE
   - Ensure example app builds and demonstrates features

6. **Backward Compatibility**:
   - Use `@Deprecated` annotations with migration paths
   - Implement graceful degradation for new features
   - Delegate legacy methods to new implementations
   - Test both old and new APIs in unit tests

### **Best Practices Checklist:**
- ✅ Use reverse domain naming for channel names to avoid collisions
- ✅ Enable null safety and validate platform responses
- ✅ Implement proper error handling with domain-specific exceptions
- ✅ Support federated architecture for multi-platform plugins
- ✅ Document platform-specific setup requirements (permissions, Info.plist, AndroidManifest.xml)
- ✅ Maintain backward compatibility across minor versions
- ✅ Write unit tests mocking platform channels
- ✅ Include integration tests in the example app

---

## **Next Steps**

In the next chapter, **Chapter 39: App Configuration & Flavors**, we will explore how to configure Flutter applications for different environments (development, staging, production) using flavors and build configurations. You'll learn how to set up environment-specific APIs, icons, and behaviors while maintaining clean separation between configurations.

---

**End of Chapter 38**